In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [2]:
df_train = pd.read_csv('../input/nlp-getting-started/train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
df_test = pd.read_csv('../input/nlp-getting-started/test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [4]:
df_train.drop(['location'], axis = 1, inplace = True)
df_train.head()

,id,keyword,text,target
0,1,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df_test.drop(['location'], axis = 1, inplace = True)
df_test.head()

,id,keyword,text
0,0,NaN,Just happened a terrible car crash
1,2,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
df_train.text = df_train.text.str.lower()
df_train.head()

,id,keyword,text,target
0,1,NaN,our deeds are the reason of this #earthquake m...,1
1,4,NaN,forest fire near la ronge sask. canada,1
2,5,NaN,all residents asked to 'shelter in place' are ...,1
3,6,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,just got sent this photo from ruby #alaska as ...,1


In [7]:
df_test.text = df_test.text.str.lower()
df_test.head()

,id,keyword,text
0,0,NaN,just happened a terrible car crash
1,2,NaN,"heard about #earthquake is different cities, s..."
2,3,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,apocalypse lighting. #spokane #wildfires
4,11,NaN,typhoon soudelor kills 28 in china and taiwan


# Mention

In [8]:
df_train['mention'] = 0
df_train.head()

,id,keyword,text,target,mention
0,1,NaN,our deeds are the reason of this #earthquake m...,1,0
1,4,NaN,forest fire near la ronge sask. canada,1,0
2,5,NaN,all residents asked to 'shelter in place' are ...,1,0
3,6,NaN,"13,000 people receive #wildfires evacuation or...",1,0
4,7,NaN,just got sent this photo from ruby #alaska as ...,1,0


In [9]:
df_train.loc[df_train.text.str.contains('@[^ ]+'),'mention'] = 1
df_train.loc[df_train.text.str.contains('@[^ ]+')]

,id,keyword,text,target,mention
31,48,ablaze,@bbcmtd wholesale markets ablaze http://t.co/l...,1,1
36,54,ablaze,@phdsquares #mufc they've built so much hype a...,0,1
43,63,ablaze,soooo pumped for ablaze ???? @southridgelife,0,1
54,78,ablaze,noches el-bestia '@alexis_sanchez: happy to se...,0,1
57,81,ablaze,set our hearts ablaze and every city was a gif...,0,1
...,...,...,...,...,...
7577,10829,wrecked,@widda16 ... he's gone. you can relax. i thoug...,0,1
7578,10830,wrecked,@jt_ruff23 @cameronhacker and i wrecked you both,0,1
7581,10833,wrecked,@engineshed great atmosphere at the british li...,0,1
7596,10851,NaN,rt @livingsafely: #nws issues severe #thunders...,1,1


In [10]:
df_test['mention'] = 0
df_test.loc[df_test.text.str.contains('@[^ ]+'),'mention'] = 1
df_test.loc[df_test.text.str.contains('@[^ ]+')]

,id,keyword,text,mention
16,47,ablaze,@sunkxssedharry will you wear shorts for race ...,1
19,60,ablaze,psa: iûªm splitting my personalities.\n\n?? t...,1
21,70,ablaze,burning man ablaze! by turban diva http://t.co...,1
22,72,ablaze,not a diss song. people will take 1 thing and ...,1
25,87,ablaze,@ctvtoronto the bins in front of the field by ...,1
...,...,...,...,...
3244,10797,wrecked,@nathan26_rfc thought you said saturday night ...,1
3246,10804,wrecked,@yakuboobs think he deactivated because his no...,1
3249,10816,wrecked,@thrillhho jsyk i haven't stopped thinking abt...,1
3250,10820,wrecked,@stighefootball begovic has been garbage. he g...,1


In [11]:
for i in df_train.loc[df_train.text.str.contains('@[^ ]')].index:
    df_train.loc[i, 'text'] = re.sub('@[^ ]', '', df_train.loc[i, 'text'])
df_train.loc[df_train.text.str.contains('@[^ ]')].count()

id         0
keyword    0
text       0
target     0
mention    0
dtype: int64

In [12]:
for i in df_test.loc[df_test.text.str.contains('@[^ ]')].index:
    df_test.loc[i, 'text'] = re.sub('@[^ ]', '', df_test.loc[i, 'text'])
df_test.loc[df_test.text.str.contains('@[^ ]')].count()

id         0
keyword    0
text       0
mention    0
dtype: int64

# URL

In [13]:
df_train['URL'] = 0
df_test['URL'] = 0

In [14]:
df_train.loc[df_train.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+'),'URL'] = 1
df_train.loc[df_train.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+')]

,id,keyword,text,target,mention,URL
31,48,ablaze,bcmtd wholesale markets ablaze http://t.co/lhy...,1,1,1
32,49,ablaze,we always try to bring the heavy. #metal #rt h...,0,0,1
33,50,ablaze,#africanbaze: breaking news:nigeria flag set a...,1,0,1
35,53,ablaze,on plus side look at the sky last night it was...,0,0,1
37,55,ablaze,inec office in abia set ablaze - http://t.co/3...,1,0,1
...,...,...,...,...,...,...
7606,10866,NaN,suicide bomber kills 15 in saudi security site...,1,0,1
7607,10867,NaN,#stormchase violent record breaking ef-5 el re...,1,0,1
7608,10869,NaN,two giant cranes holding a bridge collapse int...,1,0,1
7610,10871,NaN,m1.94 [01:04 utc]?5km s of volcano hawaii. htt...,1,0,1


In [15]:
df_test.loc[df_test.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+'),'URL'] = 1
df_test.loc[df_test.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+')]

,id,keyword,text,mention,URL
15,46,ablaze,birmingham wholesale market is ablaze bbc news...,0,1
17,51,ablaze,#previouslyondoyintv: toke makinwaûªs marriag...,0,1
18,58,ablaze,check these out: http://t.co/roi2nsmejj http:/...,0,1
21,70,ablaze,burning man ablaze! by turban diva http://t.co...,1,1
23,75,ablaze,rape victim dies as she sets herself ablaze: a...,0,1
...,...,...,...,...,...
3252,10836,NaN,#ebola #ebolaoutbreak ebola virus: birmingham ...,0,1
3253,10838,NaN,malaysian pm confirms debris is from missing f...,0,1
3255,10856,NaN,see the 16yr old pkk suicide bomber who detona...,1,1
3260,10868,NaN,green line derailment in chicago http://t.co/u...,0,1


In [16]:
for i in df_train.loc[df_train.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+')].index:
    df_train.loc[i, 'text'] = re.sub('http://[^ ]+|https://[^ ]+|www.[^ ]+', '', df_train.loc[i, 'text'])
df_train.loc[df_train.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+')].count()

id         0
keyword    0
text       0
target     0
mention    0
URL        0
dtype: int64

In [17]:
for i in df_test.loc[df_test.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+')].index:
    df_test.loc[i, 'text'] = re.sub('http://[^ ]+|https://[^ ]+|www.[^ ]+', '', df_test.loc[i, 'text'])
df_test.loc[df_test.text.str.contains('http://[^ ]+|https://[^ ]+|www.[^ ]+')].count()

id         0
keyword    0
text       0
mention    0
URL        0
dtype: int64

# Hashtag

In [18]:
a = list(df_train.text.str.findall('#[A-Za-z_]+'))
a.extend(list(df_test.text.str.findall('#[A-Za-z_]+')))

In [19]:
for i in range(len(a)):
    tmp = []
    for j in a[i]:
        tmp.append(re.sub('#', '', j))
    a[i] = tmp

In [20]:
ht_dict = {}
for i in a:
    for j in i:
        try:
            ht_dict[j] += 1
        except:
            ht_dict[j] = 1

In [21]:
ht = sorted(ht_dict, key = lambda x: ht_dict[x], reverse = True)
print(ht[:20])

['news', 'hot', 'prebreak', 'best', 'hiroshima', 'nowplaying', 'earthquake', 'gbbo', 'world', 'emmerdale', 'jobs', 'islam', 'breaking', 'hiring', 'job', 'yyc', 'japan', 'directioners', 'hailstorm', 'fukushima']


In [22]:
for i in range(20):
    print(ht[i], ht_dict[ht[i]])

news 88
hot 43
prebreak 41
best 41
hiroshima 40
nowplaying 27
earthquake 26
gbbo 23
world 20
emmerdale 19
jobs 17
islam 17
breaking 16
hiring 16
job 16
yyc 16
japan 15
directioners 14
hailstorm 14
fukushima 14


In [23]:
for i in df_train.text.str.contains('#[A-Za-z_]+').index:
    df_train.loc[i, 'text'] = re.sub('#', '', df_train.loc[i, 'text'])
df_train.head()

,id,keyword,text,target,mention,URL
0,1,NaN,our deeds are the reason of this earthquake ma...,1,0,0
1,4,NaN,forest fire near la ronge sask. canada,1,0,0
2,5,NaN,all residents asked to 'shelter in place' are ...,1,0,0
3,6,NaN,"13,000 people receive wildfires evacuation ord...",1,0,0
4,7,NaN,just got sent this photo from ruby alaska as s...,1,0,0


In [24]:
for i in df_test.text.str.contains('#[A-Za-z_]+').index:
    df_test.loc[i, 'text'] = re.sub('#', '', df_test.loc[i, 'text'])
df_test.head()

,id,keyword,text,mention,URL
0,0,NaN,just happened a terrible car crash,0,0
1,2,NaN,"heard about earthquake is different cities, st...",0,0
2,3,NaN,"there is a forest fire at spot pond, geese are...",0,0
3,9,NaN,apocalypse lighting. spokane wildfires,0,0
4,11,NaN,typhoon soudelor kills 28 in china and taiwan,0,0


# Keyword

In [25]:
keyw = set(df_train.keyword)
keyw.update(df_test.keyword)
print(keyw)

{nan, 'ruin', 'drought', 'obliterate', 'panicking', 'riot', 'seismic', 'crushed', 'survive', 'demolish', 'forest%20fires', 'bloody', 'derail', 'derailment', 'fatality', 'lightning', 'hijacker', 'wounded', 'weapons', 'buildings%20burning', 'detonate', 'famine', 'burning', 'earthquake', 'windstorm', 'terrorism', 'crash', 'bioterror', 'first%20responders', 'war%20zone', 'oil%20spill', 'refugees', 'rescue', 'flooding', 'mudslide', 'deaths', 'crashed', 'emergency%20services', 'stretcher', 'destroyed', 'rioting', 'armageddon', 'hazard', 'attacked', 'storm', 'injuries', 'injured', 'avalanche', 'heat%20wave', 'hijack', 'danger', 'fatalities', 'police', 'damage', 'wild%20fires', 'snowstorm', 'sandstorm', 'deluge', 'mayhem', 'eyewitness', 'bush%20fires', 'fear', 'epicentre', 'trouble', 'screams', 'burning%20buildings', 'evacuate', 'dead', 'cyclone', 'screaming', 'inundated', 'hostage', 'body%20bagging', 'dust%20storm', 'sinkhole', 'arsonist', 'battle', 'mass%20murder', 'rubble', 'natural%20disas

In [26]:
print(len(keyw))

222


In [27]:
keyw_dict = {}
for i in keyw:
    keyw_dict[i] = 0

In [28]:
for i in df_train.keyword:
    keyw_dict[i] += 1
for i in df_test.keyword:
    keyw_dict[i] += 1

In [29]:
keyw = sorted(keyw_dict, key = lambda x: keyw_dict[x], reverse = True)
for i in range(20):
    print(keyw[i], keyw_dict[keyw[i]])

nan 87
ruin 50
drought 50
obliterate 50
panicking 50
riot 50
seismic 50
crushed 50
survive 50
demolish 50
forest%20fires 50
bloody 50
derail 50
derailment 50
fatality 50
lightning 50
hijacker 50
wounded 50
weapons 50
buildings%20burning 50


In [30]:
for i in range(-1, -20, -1):
    print(keyw[i], keyw_dict[keyw[i]])

epicentre 13
radiation%20emergency 14
inundation 14
threat 16
forest%20fire 24
battle 33
rescue 33
war%20zone 35
bush%20fires 38
volcano 42
bombing 42
chemical%20emergency 45
desolate 45
mass%20murderer 46
collision 49
airplane%20accident 50
blizzard 50
derailed 50
demolished 50


In [31]:
for i in range(9):
    if ht[i] not in keyw:
        keyw.append(ht[i])

In [32]:
print(len(keyw))

230


In [33]:
df_keyword = pd.DataFrame([[0 for i in range(230)] for i in range(len(df_train))])
df_keyword.columns = keyw
df_keyword.head()

,NaN,ruin,drought,obliterate,panicking,riot,seismic,crushed,survive,demolish,...,radiation%20emergency,epicentre,news,hot,prebreak,best,hiroshima,nowplaying,gbbo,world
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df_keyword.drop([keyw[0]], axis = 1, inplace = True)
df_keyword.head()

,ruin,drought,obliterate,panicking,riot,seismic,crushed,survive,demolish,forest%20fires,...,radiation%20emergency,epicentre,news,hot,prebreak,best,hiroshima,nowplaying,gbbo,world
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
test_keyword = pd.DataFrame([[0 for i in range(230)] for i in range(len(df_test))])
test_keyword.columns = keyw
test_keyword.head()

,NaN,ruin,drought,obliterate,panicking,riot,seismic,crushed,survive,demolish,...,radiation%20emergency,epicentre,news,hot,prebreak,best,hiroshima,nowplaying,gbbo,world
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
test_keyword.drop([keyw[0]], axis = 1, inplace = True)
test_keyword.head()

,ruin,drought,obliterate,panicking,riot,seismic,crushed,survive,demolish,forest%20fires,...,radiation%20emergency,epicentre,news,hot,prebreak,best,hiroshima,nowplaying,gbbo,world
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df_train.keyword = df_train.keyword.fillna('NaN')
for i in df_train.index:
    if df_train.loc[i, 'keyword'] != 'NaN':
        df_keyword.loc[i, df_train.loc[i, 'keyword']] = 1
df_keyword.sum()

ruin          37
drought       35
obliterate    31
panicking     33
riot          34
              ..
best           0
hiroshima      0
nowplaying     0
gbbo           0
world          0
Length: 229, dtype: int64

In [38]:
df_test.keyword = df_test.keyword.fillna('NaN')
for i in df_test.index:
    if df_test.loc[i, 'keyword'] != 'NaN':
        test_keyword.loc[i, df_test.loc[i, 'keyword']] = 1
test_keyword.sum()

ruin          13
drought       15
obliterate    19
panicking     17
riot          16
              ..
best           0
hiroshima      0
nowplaying     0
gbbo           0
world          0
Length: 229, dtype: int64

In [40]:
for i in df_keyword.columns:
    df_keyword.loc[df_train[df_train.text.str.contains(i) == True].index, i] = 1
    test_keyword.loc[df_test[df_test.text.str.contains(i) == True].index, i] = 1

In [41]:
df_keyword.head()

,ruin,drought,obliterate,panicking,riot,seismic,crushed,survive,demolish,forest%20fires,...,radiation%20emergency,epicentre,news,hot,prebreak,best,hiroshima,nowplaying,gbbo,world
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [42]:
df_keyword.sum()    

ruin           44
drought        41
obliterate     62
panicking      33
riot           85
             ... 
best           98
hiroshima      90
nowplaying     25
gbbo           21
world         140
Length: 229, dtype: int64

In [43]:
test_keyword.sum()

ruin          16
drought       19
obliterate    37
panicking     17
riot          30
              ..
best          46
hiroshima     48
nowplaying     4
gbbo           4
world         50
Length: 229, dtype: int64

In [44]:
df_train.drop(['keyword'], axis = 1, inplace = True)
df_train.head()

,id,text,target,mention,URL
0,1,our deeds are the reason of this earthquake ma...,1,0,0
1,4,forest fire near la ronge sask. canada,1,0,0
2,5,all residents asked to 'shelter in place' are ...,1,0,0
3,6,"13,000 people receive wildfires evacuation ord...",1,0,0
4,7,just got sent this photo from ruby alaska as s...,1,0,0


In [45]:
df_test.drop(['keyword'], axis = 1, inplace = True)
df_test.head()

,id,text,mention,URL
0,0,just happened a terrible car crash,0,0
1,2,"heard about earthquake is different cities, st...",0,0
2,3,"there is a forest fire at spot pond, geese are...",0,0
3,9,apocalypse lighting. spokane wildfires,0,0
4,11,typhoon soudelor kills 28 in china and taiwan,0,0


In [46]:
df_keyword['mention'] = df_train['mention']
df_keyword['URL'] = df_train['URL']
df_keyword.head()

,ruin,drought,obliterate,panicking,riot,seismic,crushed,survive,demolish,forest%20fires,...,news,hot,prebreak,best,hiroshima,nowplaying,gbbo,world,mention,URL
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [47]:
test_keyword['mention'] = df_test['mention']
test_keyword['URL'] = df_test['URL']
test_keyword.head()

,ruin,drought,obliterate,panicking,riot,seismic,crushed,survive,demolish,forest%20fires,...,news,hot,prebreak,best,hiroshima,nowplaying,gbbo,world,mention,URL
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Words

In [48]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stopw = set(stopwords.words('english'))
stopw.update(['\'m'])
wordlist = {}
for i in df_train.text:
    w = word_tokenize(i)
    for j in w:
        if j in stopw:
            continue
        try:
            wordlist[j] += 1
        except:
            wordlist[j] = 1
for i in df_test.text:
    w = word_tokenize(i)
    for j in w:
        if j in stopw:
            continue
        try:
            wordlist[j] += 1
        except:
            wordlist[j] = 1
print(len(wordlist))

23476


In [49]:
wl = sorted(wordlist, key = lambda x:wordlist[x], reverse = True)
for i in range(20):
    print(wl[i], wordlist[wl[i]])

? 4446
. 4221
: 2847
! 1666
... 1530
's 1117
- 1092
' 941
; 835
& 651
n't 621
) 529
amp 510
( 504
like 489
fire 356
get 333
via 322
new 320
people 277


In [50]:
cnt = 0
for i in wl:
    if wordlist[i] < 20:
        cnt += 1
print(cnt)

22408


In [51]:
tmp = []
for i in wl:
    if wordlist[i] >= 20:
        tmp.append(i)

In [52]:
vocab = {}
for i in range(len(tmp)):
    vocab[tmp[i]] = i

In [53]:
txt = []
for i in df_train.text:
    w = word_tokenize(i)
    tmp = []
    for j in w:
        if j in stopw:
            continue
        try:
            tmp.append(vocab[j])
        except:
            tmp.append(len(vocab))
    txt.append(tmp)
print(txt[:5])

[[1068, 720, 180, 84, 1068, 1068, 34], [97, 15, 135, 823, 1068, 1068, 1, 1068], [1068, 1068, 1068, 621, 7, 1068, 1068, 1, 183, 1068, 621, 1068, 1068], [1068, 19, 1068, 1068, 183, 1068, 46], [33, 1068, 207, 1068, 1068, 189, 1068, 1068, 130]]


In [54]:
df_txt = pd.DataFrame(txt)
df_txt.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1068,720.0,180.0,84.0,1068.0,1068.0,34.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97,15.0,135.0,823.0,1068.0,1068.0,1.0,1068.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1068,1068.0,1068.0,621.0,7.0,1068.0,1068.0,1.0,183.0,1068.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1068,19.0,1068.0,1068.0,183.0,1068.0,46.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33,1068.0,207.0,1068.0,1068.0,189.0,1068.0,1068.0,130.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
txt = []
for i in df_test.text:
    w = word_tokenize(i)
    tmp = []
    for j in w:
        if j in stopw:
            continue
        try:
            tmp.append(vocab[j])
        except:
            tmp.append(len(vocab))
    txt.append(tmp)
print(txt[:5])

[[666, 1068, 69, 39], [275, 180, 1068, 1068, 1068, 539, 1068, 173, 1], [97, 15, 708, 1068, 1068, 1068, 1068, 766, 569, 1068, 373], [287, 1068, 1, 1068, 1068], [438, 618, 255, 1068, 360, 1068]]


In [56]:
test_txt = pd.DataFrame(txt)
test_txt.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,666,1068.0,69.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,275,180.0,1068.0,1068.0,1068.0,539.0,1068.0,173.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97,15.0,708.0,1068.0,1068.0,1068.0,1068.0,766.0,569.0,1068.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,287,1068.0,1.0,1068.0,1068.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,438,618.0,255.0,1068.0,360.0,1068.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
print(len(df_txt))
print(df_txt[25].isnull().sum())

7613
7571


In [58]:
print(len(test_txt))
print(test_txt[25].isnull().sum())

3263
3244


In [59]:
df_txt.drop(df_txt.columns[26:], axis = 1, inplace = True)
df_txt.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1068,720.0,180.0,84.0,1068.0,1068.0,34.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97,15.0,135.0,823.0,1068.0,1068.0,1.0,1068.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1068,1068.0,1068.0,621.0,7.0,1068.0,1068.0,1.0,183.0,1068.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1068,19.0,1068.0,1068.0,183.0,1068.0,46.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33,1068.0,207.0,1068.0,1068.0,189.0,1068.0,1068.0,130.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
test_txt.drop(test_txt.columns[26:], axis = 1, inplace = True)
test_txt.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,666,1068.0,69.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,275,180.0,1068.0,1068.0,1068.0,539.0,1068.0,173.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97,15.0,708.0,1068.0,1068.0,1068.0,1068.0,766.0,569.0,1068.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,287,1068.0,1.0,1068.0,1068.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,438,618.0,255.0,1068.0,360.0,1068.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_txt = df_txt.fillna(len(vocab)+1)
df_txt = df_txt.astype(int)
df_txt.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1068,720,180,84,1068,1068,34,1069,1069,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
1,97,15,135,823,1068,1068,1,1068,1069,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
2,1068,1068,1068,621,7,1068,1068,1,183,1068,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
3,1068,19,1068,1068,183,1068,46,1069,1069,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
4,33,1068,207,1068,1068,189,1068,1068,130,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069


In [62]:
test_txt = test_txt.fillna(len(vocab)+1)
test_txt = test_txt.astype(int)
test_txt.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,666,1068,69,39,1069,1069,1069,1069,1069,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
1,275,180,1068,1068,1068,539,1068,173,1,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
2,97,15,708,1068,1068,1068,1068,766,569,1068,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
3,287,1068,1,1068,1068,1069,1069,1069,1069,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
4,438,618,255,1068,360,1068,1069,1069,1069,1069,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069


# Training

In [63]:
np_keyword = np.array(df_keyword)
test_keyword = np.array(test_keyword)
print(np.shape(np_keyword))
print(np.shape(test_keyword))

(7613, 231)
(3263, 231)


In [64]:
np_txt = np.array(df_txt)
test_txt = np.array(test_txt)
print(np.shape(np_txt))
print(np.shape(test_txt))

(7613, 26)
(3263, 26)


In [65]:
y_train = np.array(df_train['target'])
y_train = y_train.reshape(-1, 1)
np.shape(y_train)

(7613, 1)

In [66]:
from keras import models, layers
from keras.callbacks import EarlyStopping

txt = layers.Input((26,))
txt_h = layers.Embedding(len(vocab)+2, 128)(txt)
txt_h = layers.LSTM(30)(txt_h)
keyw = layers.Input((231,))
keyw_h = layers.Dense(128)(keyw)
h = layers.concatenate([txt_h, keyw_h], axis = -1)
h = layers.Dense(64)(h)
h = layers.Dense(32)(h)
o = layers.Dense(1, activation = 'sigmoid')(h)

model = models.Model([txt, keyw], o)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_accuracy', mode = 'max', restore_best_weights = True, patience = 5)
model.fit([np_txt, np_keyword], y_train, epochs = 1000, batch_size = 50, callbacks = [early_stopping], validation_split = 0.2)

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/1000
6090/6090 [==============================] - 3s 519us/step - loss: 0.5386 - accuracy: 0.7355 - val_loss: 0.4775 - val_accuracy: 0.7682
Epoch 2/1000
6090/6090 [==============================] - 2s 349us/step - loss: 0.4364 - accuracy: 0.8102 - val_loss: 0.4635 - val_accuracy: 0.7774
Epoch 3/1000
6090/6090 [==============================] - 2s 349us/step - loss: 0.4168 - accuracy: 0.8190 - val_loss: 0.4586 - val_accuracy: 0.7997
Epoch 4/1000
6090/6090 [==============================] - 2s 362us/step - loss: 0.4048 - accuracy: 0.8251 - val_loss: 0.4553 - val_accuracy: 0.7866
Epoch 5/1000
6090/6090 [==============================] - 3s 431us/step - loss: 0.3955 - accuracy: 0.8350 - val_loss: 0.4620 - val_accuracy: 0.7800
Epoch 6/1000
6090/6090 [==============================] - 3s 469us/step - loss: 0.3887 - accuracy: 0.8348 - val_loss: 0.5003 - val_accuracy: 0.7472
Epoch 7/1000
6090/6090 [==============================] - 3s 427

# Predict

In [67]:
result = model.predict([test_txt, test_keyword])
print(result)

[[0.5237808]
 [0.6514936]
 [0.5102394]
 ...
 [0.9408398]
 [0.7683238]
 [0.3210827]]


In [68]:
pred = []
for i in result:
    pred.append(round(float(i)))
print(len(pred))

3263


In [69]:
sample = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
sample.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [70]:
submission = pd.DataFrame()
submission['id'] = df_test['id']
submission.head()

,id
0,0
1,2
2,3
3,9
4,11


In [71]:
submission['target'] = pred
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [72]:
submission.to_csv('submission.csv', index = False)